## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[pillow, tqdm]"
!python -c "import matplotlib" || pip install -q matplotlib
!pip install scipy
%matplotlib inline

## Setup imports

In [ ]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import tempfile
import matplotlib.pyplot as plt
import PIL
import torch
import numpy as np
from sklearn.metrics import classification_report

from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import decollate_batch, DataLoader
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
from monai.networks.nets import ResNet
from monai.networks.nets import EfficientNetBN
from monai.transforms import (
    Activations,
    EnsureChannelFirst,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
)
from monai.utils import set_determinism

print_config()

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [ ]:
# Especificar un directorio con la variable de entorno MONAI_DATA_DIRECTORY,
# permitiendo esto guardar los resultados y reutilizar las descargas.
# Si no se especifica, se utilizará un directorio temporal.

directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

In [ ]:
# Este código gestiona la descarga y extracción del conjutno de datos (MedNIST.tar.gz) desde una URL específica.
# Se verifica la integridad del archivo mediante su hash MD5 antes de la descarga.
# El recurso se almacena en un archivo comprimido en el directorio raíz y se extrae en un directorio de datos.
# La descarga y extracción solo se realizan si el directorio de datos aún no existe.
resource = "https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/MedNIST.tar.gz"
md5 = "0bc7306e7427e00ad1c5526a6677552d"

compressed_file = os.path.join(root_dir, "MedNIST.tar.gz")
data_dir = os.path.join(root_dir, "MedNIST")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

## Set deterministic training for reproducibility

In [ ]:
# Establecer el determinismo en el código, utilizando una semilla (seed) específica.
# Esto garantiza que las operaciones aleatorias generadas en el código sean reproducibles, siempre y cuando
# se utilice la misma semilla. Útil para la reproducibilidad de resultados en tareas como entrenamiento de modelos.

set_determinism(seed=0)

## Read image filenames from the dataset folders

First of all, check the dataset files and show some statistics.  
There are 6 folders in the dataset: Hand, AbdomenCT, CXR, ChestCT, BreastMRI, HeadCT,  
which should be used as the labels to train our classification model.

In [ ]:
# Obtener la lista ordenada de nombres de clases a partir de los subdirectorios en el directorio de datos
class_names = sorted(x for x in os.listdir(data_dir)
                     if os.path.isdir(os.path.join(data_dir, x)))

# Calcular el número total de clases
num_class = len(class_names)

# Obtener la lista de rutas de archivos de imagen para cada clase
image_files = [
    [
        os.path.join(data_dir, class_names[i], x)
        for x in os.listdir(os.path.join(data_dir, class_names[i]))
    ]
    for i in range(num_class)
]

#Calcular el número de imágenes en cada clase
num_each = [len(image_files[i]) for i in range(num_class)]

# Crear una lista plana de rutas de archivos de imagen y una lista de etiquetas correspondientes
image_files_list = []
image_class = []
for i in range(num_class):
    image_files_list.extend(image_files[i])
    image_class.extend([i] * num_each[i])

# Calcular el número total de imágenes
num_total = len(image_class)

# Obtener las dimensiones de la primera imagen
image_width, image_height = PIL.Image.open(image_files_list[0]).size

print(PIL.Image.open(image_files_list[0]))
print(f"Total image count: {num_total}")
print(f"Image dimensions: {image_width} x {image_height}")
print(f"Label names: {class_names}")
print(f"Label counts: {num_each}")

## Randomly pick images from the dataset to visualize and check

In [ ]:
# Este código carga y muestra 10 imágenes en una cuadrícula de 2 filas y 5 columnas.

plt.subplots(2, 5, figsize=(8, 8))
for i, k in enumerate(np.random.randint(num_total, size=10)):
    im = PIL.Image.open(image_files_list[k])
    arr = np.array(im)
    plt.subplot(2, 5, i + 1)
    plt.xlabel(class_names[image_class[k]])
    plt.imshow(arr, vmin=0, vmax=255)
plt.tight_layout()
plt.show()

## Prepare training, validation and test data lists

Randomly select 10% of the dataset as validation and 10% as test.

In [ ]:
# Dividir los índices de las imágenes en conjuntos de entrenamiento, validación y test
# utilizando porcentajes porcentajes para cada uno de los conjuntos
val_frac = 0.1
test_frac = 0.1
length = len(image_files_list)
indices = np.arange(length)
np.random.shuffle(indices)

# Calcular los índices para cada conjunto según los porcertajes especificados
test_split = int(test_frac * length)
val_split = int(val_frac * length) + test_split
test_indices = indices[:test_split]
val_indices = indices[test_split:val_split]
train_indices = indices[val_split:]

# Crear listas de rutas de archivos y etiquetas para cada conjunto
train_x = [image_files_list[i] for i in train_indices]
train_y = [image_class[i] for i in train_indices]
val_x = [image_files_list[i] for i in val_indices]
val_y = [image_class[i] for i in val_indices]
test_x = [image_files_list[i] for i in test_indices]
test_y = [image_class[i] for i in test_indices]

print(
    f"Training count: {len(train_x)}, Validation count: "
    f"{len(val_x)}, Test count: {len(test_x)}")

## Define MONAI transforms, Dataset and Dataloader to pre-process data

In [ ]:
# Definir transformaciones de datos para los conjuntos de entrenamiento, validación y predicciones

# Transformaciones para el conjunto de entrenamiento
train_transforms = Compose(
    [
        LoadImage(image_only=True), # Cargar la imagen
        EnsureChannelFirst(), # Asegurar que los canales estén en la primera dimensión
        ScaleIntensity(), # Escalar la intensidad de los píxeles
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True), # Rotación aleatoria
        RandFlip(spatial_axis=0, prob=0.5), # Volteo aleatorio en el eje espacial 0 (horizontal)
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5), # Zoom aleatorio
    ]
)

# Transformaciones para el conjunto de validación
val_transforms = Compose(
    [LoadImage(image_only=True), EnsureChannelFirst(), ScaleIntensity()])

# Transformaciones para las predicciones
y_pred_trans = Compose([Activations(softmax=True)]) # Aplicar funciones de activación softmax

# Transformaciones para las etiquetas
y_trans = Compose([AsDiscrete(to_onehot=num_class)]) # Convertir a representación one-hot

In [ ]:
# Definir la clase MedNISTDataset que hereda de torch.utils.data.Dataset.
# Esta clase se utiliza para crear los conjuntos de datos (entrenamiento, validación y test),
# que contienen rutas de archivos de imágenes y etiquetas correspondientes, así como
# las transformaciones que se aplicarán durante la evaluación.

class MedNISTDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]

workers = 2

# Crear un conjunto de datos de entrenamiento utilizando la clase MedNISTDataset
# y configurar un cargador de datos utilizando DataLoader de PyTorch.
train_ds = MedNISTDataset(train_x, train_y, train_transforms)
train_loader = DataLoader(
    train_ds, batch_size=100, shuffle=True, num_workers=workers)

# Crear un conjunto de datos de validación utilizando la clase MedNISTDataset
# y configurar un cargador de datos utilizando DataLoader de PyTorch.
val_ds = MedNISTDataset(val_x, val_y, val_transforms)
val_loader = DataLoader(
    val_ds, batch_size=100, num_workers=workers)

# Crear un conjunto de datos de test utilizando la clase MedNISTDataset
# y configurar un cargador de datos utilizando DataLoader de PyTorch.
test_ds = MedNISTDataset(test_x, test_y, val_transforms)
test_loader = DataLoader(
    test_ds, batch_size=100, num_workers=workers)

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from scipy.spatial.distance import cdist

In [ ]:
class GCNN_ECNN(nn.Module):
    def __init__(self, img_size=64, pred_edge=False):
        super(GCNN_ECNN, self).__init__()

        # Establecer el indicador de predicción de bordes
        self.pred_edge = pred_edge
        # Calcular el tamaño de la entrada para la capa completamente conectada
        N = img_size ** 2

        # Capa completamente conectada que toma la entrada de tamaño N y produce una salida de 6 clases
        self.fc = nn.Linear(N, 6, bias=True)

        # Verificar si se debe realizar la predicción de bordes
        if pred_edge:
            # Preparar coordenadas normalizadas para la predicción de bordes
            col, row = np.meshgrid(np.arange(img_size), np.arange(img_size))
            coord = np.stack((col, row), axis=2).reshape(-1, 2)
            coord = (coord - np.mean(coord, axis=0)) / (np.std(coord, axis=0) + 1e-5)
            coord = torch.from_numpy(coord).float()  # 784,2
            # Expandir las coordenadas para su uso en la red neuronal
            coord = torch.cat((coord.unsqueeze(0).repeat(N, 1,  1),
                                    coord.unsqueeze(1).repeat(1, N, 1)), dim=2)
            #coord = torch.abs(coord[:, :, [0, 1]] - coord[:, :, [2, 3]]) # this should have worked
            # Definir la red neuronal para la predicción de bordes
            self.pred_edge_fc = nn.Sequential(nn.Linear(4, 32), # Esto puede ser un hiperparámetro
                                              nn.ReLU(),
                                              nn.Linear(32, 64),
                                              nn.ReLU(),
                                              nn.Linear(64, 1),
                                              nn.Tanh())
            # Registrar las coordenadas como un tensor constante
            self.register_buffer('coord', coord)

        # Si no se realiza predicción de bordes
        else:
            # Precomputar y registrar la matriz de adyacencia para la capa GCN
            A = self.precompute_adjacency_images(img_size)
            self.register_buffer('A', A)


    @staticmethod
    def precompute_adjacency_images(img_size):

        # Crear malla de coordenadas (col, row) para la imagen de tamaño img_size
        col, row = np.meshgrid(np.arange(img_size), np.arange(img_size))
        # Apilar las coordenadas (col, row) y normalizarlas
        coord = np.stack((col, row), axis=2).reshape(-1, 2) / img_size
        # Calcular la matriz de distancias euclidianas entre las coordenadas
        dist = cdist(coord, coord)
        # Parámetro de suavizado para la función de densidad gaussiana
        sigma = 0.05 * np.pi

        # Calcular la matriz de adyacencia usando una función de densidad gaussiana
        A = np.exp(- dist / sigma ** 2)
        print('WARNING: try squaring the dist to make it a Gaussian')

        # Filtrar los elementos pequeños de la matriz de adyacencia y convertirla a tensor de PyTorch
        A[A < 0.01] = 0
        A = torch.from_numpy(A).float()

        # Normalización según (Kipf & Welling, ICLR 2017)
        D = A.sum(1)  # Grados de los nodos (N,)
        D_hat = (D + 1e-5) ** (-0.5)
        A_hat = D_hat.view(-1, 1) * A * D_hat.view(1, -1)  # N,N

        # Truco adicional encontrado útil
        A_hat[A_hat > 0.0001] = A_hat[A_hat > 0.0001] - 0.2

        # print(A_hat[:10, :10])
        return A_hat

    def forward(self, x):
        B = x.size(0)

        # Actualizar la matriz de adyacencia si se está prediciendo los bordes
        if self.pred_edge:
            self.A = self.pred_edge_fc(self.coord).squeeze()

        # Calcular características promedio de los vecinos utilizando la matriz de adyacencia
        avg_neighbor_features = (torch.bmm(self.A.unsqueeze(0).expand(B, -1, -1),
                                 x.view(B, -1, 1)).view(B, -1))

        # Pasar las características promedio a la siguiente capa
        return self.fc(avg_neighbor_features)

In [ ]:
# Crear una instancia del modelo GCNN_ECNN con la opción de predicción de bordes habilitada
model = GCNN_ECNN(pred_edge=True)

In [ ]:
# Imprimir la cantidad total de parámetros en el modelo
print(sum(p.numel() for p in model.parameters()))

## Define network and optimizer

1. Set learning rate for how much the model is updated per batch.
1. Set total epoch number, as we have shuffle and random transforms, so the training data of every epoch is different.  
And as this is just a get start tutorial, let's just train 4 epochs.  
If train 10 epochs, the model can achieve 100% accuracy on test dataset.
1. Use DenseNet from MONAI and move to GPU devide, this DenseNet can support both 2D and 3D classification tasks.
1. Use Adam optimizer.

In [ ]:
# Determina el dispositivo de ejecución
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define como función de pérdida la entropía cruzada
loss_function = torch.nn.CrossEntropyLoss()
# Crea un optimizador Adam para ajustar los parámetros del modelo
# con una tasa de aprendizaje de 1e-5
optimizer = torch.optim.Adam(model.parameters(), 1e-5)
# Número máximo de épocas para el entrenamiento
max_epochs = 50
# Intervalo para la validación (cada cuántas épocas se realiza)
val_interval = 1
# Crea una métrica de área bajo la curva ROC para evaluar el rendimiento
auc_metric = ROCAUCMetric()
# Mueve el modelo a la GPU (si está disponible)
model.to(device)

In [ ]:
import time

## Model training

Execute a typical PyTorch training that run epoch loop and step loop, and do validation after every epoch.  
Will save the model weights to file if got best validation accuracy.

In [ ]:
# Variables para el seguimiento de la mejor métrica
best_metric = -1
best_metric_epoch = -1
# Listas para almacenar los valores de pérdida y métrica en cada época
epoch_loss_values = []
metric_values = []

# Bucle de entrenamiento
for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")

    # Configura el modelo en modo de entrenamiento
    model.train()
    epoch_loss = 0
    step = 0
    for_time = []
    back_time = []

    # Itera sobre lotes de datos de entrenamiento
    for batch_data in train_loader:
        # print(batch_data[0].shape)
        step += 1
        # Mueve los datos de entrada a la GPU o CPU
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        # Inicializa los gradientes a cero
        optimizer.zero_grad()
        # Realiza la propagación hacia adelante (forward)
        a = time.time()
        outputs = model(inputs)
        # Calcula la pérdida
        loss = loss_function(outputs, labels)
        b = time.time()
        # Realiza la retropropagación y actualiza los pesos del modelo
        loss.backward()
        c = time.time()
        optimizer.step()
        d = time.time()

        # Registro de los tiempos de ejecución
        for_time.append(b-a)
        #print("forward",sum(for_time)/len(for_time))
        epoch_loss += loss.item()
        back_time.append(c-b)
        # print("backward",sum(back_time)/len(back_time))

        print(
            f"{step}/{len(train_ds) // train_loader.batch_size}, "
            f"train_loss: {loss.item():.4f}")
        epoch_len = len(train_ds) // train_loader.batch_size

    # Cálculo de la pérdida promedio
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    # Validación cada cierto número de épocas
    if (epoch + 1) % val_interval == 0:
        model.eval()
        # Validación sobre el conjunto de validación
        with torch.no_grad():
            y_pred = torch.tensor([], dtype=torch.float32, device=device)
            y = torch.tensor([], dtype=torch.long, device=device)
            for val_data in val_loader:
                val_images, val_labels = (
                    val_data[0].to(device),
                    val_data[1].to(device),
                )
                y_pred = torch.cat([y_pred, model(val_images)], dim=0)
                y = torch.cat([y, val_labels], dim=0)

            # Evalúa la métrica AUC-ROC en el conjunto de validación
            y_onehot = [y_trans(i) for i in decollate_batch(y, detach=False)]
            y_pred_act = [y_pred_trans(i) for i in decollate_batch(y_pred)]
            auc_metric(y_pred_act, y_onehot)
            result = auc_metric.aggregate()
            auc_metric.reset()
            del y_pred_act, y_onehot
            metric_values.append(result)

            # Calcula y almacena la precisión en el conjunto de validación
            acc_value = torch.eq(y_pred.argmax(dim=1), y)
            acc_metric = acc_value.sum().item() / len(acc_value)
            # Actualiza el modelo si se obtiene una métrica mejor
            if result > best_metric:
                best_metric = result
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), os.path.join(
                    root_dir, "best_metric_model.pth"))
                print("saved new best metric model")
            # Información sobre la métrica actual y la mejor métrica alcanzada
            print(
                f"current epoch: {epoch + 1} current AUC: {result:.4f}"
                f" current accuracy: {acc_metric:.4f}"
                f" best AUC: {best_metric:.4f}"
                f" at epoch: {best_metric_epoch}"
            )

# Mensaje indicando que el entrenamiento ha sido completado
print(
    f"train completed, best_metric: {best_metric:.4f} "
    f"at epoch: {best_metric_epoch}")

## Plot the loss and metric

In [ ]:
# Configuración de la figura
plt.figure("train", (12, 6))
# Pérdida promedio por época
plt.subplot(1, 2, 1)
plt.title("Epoch Average Loss")
x = [i + 1 for i in range(len(epoch_loss_values))]
y = epoch_loss_values
plt.xlabel("epoch")
plt.plot(x, y)
# Subtrama derecha: Métrica AUC en el conjunto de validación
plt.subplot(1, 2, 2)
plt.title("Val AUC")
x = [val_interval * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel("epoch")
plt.plot(x, y)
# Muestra la figura
plt.show()

## Evaluate the model on test dataset

After training and validation, we already got the best model on validation test.  
We need to evaluate the model on test dataset to check whether it's robust and not over-fitting.  
We'll use these predictions to generate a classification report.

In [ ]:
# Carga los pesos del modelo con la mejor métrica
model.load_state_dict(torch.load(
    os.path.join(root_dir, "best_metric_model.pth")))

# Entrar en modo evaluación
model.eval()
# Listas para almacenar las etiquetas verdaderas y las predicciones del modelo
y_true = []
y_pred = []
# Realiza la evaluación en el conjunto de prueba
with torch.no_grad():
    for test_data in test_loader:
        test_images, test_labels = (
            test_data[0].to(device),
            test_data[1].to(device),
        )
        # Obtención las predicciones del modelo
        pred = model(test_images).argmax(dim=1)
        # Almacena las etiquetas verdaderas y las predicciones
        for i in range(len(pred)):
            y_true.append(test_labels[i].item())
            y_pred.append(pred[i].item())

In [ ]:
# Informe de clasificación
print(classification_report(
    y_true, y_pred, target_names=class_names, digits=4))

## Uncertainty

In [ ]:
length = len(image_files_list)
indices = np.arange(length)
np.random.shuffle(indices)

split = []
split_prediction = []
last_split = 0
# Asignación de los índices a cada subconjunto
for k in range(1,11):
    new_split = int(0.1 * k * length)
    new_split_indices = indices[last_split:new_split]
    split.append([image_files_list[i] for i in new_split_indices])
    split_prediction.append([image_class[i] for i in new_split_indices])
    last_split = new_split


auc = 0
metric_values = []
workers = 2

# Iteración sobre cada uno de los subconjuntos
for i in range(10):
    metric_values.append([])

    # Obtención de los valores de los índices
    resultado_x = [elemento for indice, elemento in enumerate(split) if indice != i]
    resultado_y = [elemento for indice, elemento in enumerate(split_prediction) if indice != i]
    # Combina todas las listas en una sola
    split_train = [elemento for sublista in resultado_x for elemento in sublista]
    split_prediction_train = [elemento for sublista in resultado_y for elemento in sublista]

    # Creación de conjunto de entrenamiento y validación
    train_ds = MedNISTDataset(split_train, split_prediction_train, train_transforms)
    train_loader = DataLoader(
        train_ds, batch_size=100, shuffle=True, num_workers=workers)

    test_ds = MedNISTDataset(split[i], split_prediction[i], val_transforms)
    test_loader = DataLoader(
        test_ds, batch_size=100, num_workers=workers)


    # Model
    model = GCNN_ECNN(pred_edge=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), 1e-5)
    max_epochs = 4
    auc_metric = ROCAUCMetric()
    model.to(device)

    epoch_loss_values = []
    # Bucle de entrenamiento
    for epoch in range(max_epochs):
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")

        # Configura el modelo en modo de entrenamiento
        model.train()
        epoch_loss = 0
        step = 0
        for_time = []
        back_time = []

        # Itera sobre lotes de datos de entrenamiento
        for batch_data in train_loader:
            step += 1
            # Mueve los datos de entrada a la GPU o CPU
            inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
            # Inicializa los gradientes a cero
            optimizer.zero_grad()
            # Realiza la propagación hacia adelante (forward)
            a = time.time()
            outputs = model(inputs)
            # Calcula la pérdida
            loss = loss_function(outputs, labels)
            b = time.time()
            # Realiza la retropropagación y actualiza los pesos del modelo
            loss.backward()
            c = time.time()
            optimizer.step()
            d = time.time()

            # Registro de los tiempos de ejecución
            for_time.append(b-a)
            epoch_loss += loss.item()
            back_time.append(c-b)
            print(
                f"{step}/{len(train_ds) // train_loader.batch_size}, "
                f"train_loss: {loss.item():.4f}")
            epoch_len = len(train_ds) // train_loader.batch_size
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        # Validación
        model.eval()
        y_true = []
        y_pred = []
        with torch.no_grad():
            y_pred = torch.tensor([], dtype=torch.float32, device=device)
            y = torch.tensor([], dtype=torch.long, device=device)
            for test_data in test_loader:
                val_images, val_labels = (
                    test_data[0].to(device),
                    test_data[1].to(device),
                )
                y_pred = torch.cat([y_pred, model(val_images)], dim=0)
                y = torch.cat([y, val_labels], dim=0)

            # Evalúa la métrica AUC-ROC en el conjunto de validación
            y_onehot = [y_trans(i) for i in decollate_batch(y, detach=False)]
            y_pred_act = [y_pred_trans(i) for i in decollate_batch(y_pred)]
            auc_metric(y_pred_act, y_onehot)
            metric_values[i].append(auc_metric.aggregate())
            auc_metric.reset()
            del y_pred_act, y_onehot

In [ ]:
import statistics
import math
import numpy as np
import scipy.stats as st

# Resultados obtenidos
print("Todos los AUC: ", metric_values)
metric_transpose = np.array(metric_values).transpose()
for i in range(len(metric_transpose)):
    gfg_data = metric_transpose[i]
    mean = np.mean(gfg_data)
    dev = statistics.stdev(gfg_data)
    sem = dev/math.sqrt(10)
    conf = st.t.interval(confidence=0.95, df=len(gfg_data)-1, loc=np.mean(gfg_data), scale=sem)
    conf = (round(conf[0], 3), round(conf[1], 3))
    print("Época ", i+1, ":")
    print("----------")
    print("Media: ", round(mean,  3))
    print("Desviación estándar: ", round(dev, 3))
    print("SEM: ", round(sem, 3))
    print("Intervalo de confianza: ", conf)

## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)